In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import widgets
import time
from IPython import display
import numpy as np

#widgets for user's information

LabelHi = widgets.Text(value="Hi, before starting we have to ask you something about you", disabled=True)
LabelHi.background_color = "#FFFFFF"
LabelHi.margin = "20px"
display.display(LabelHi)

Name=widgets.Text(description="Name")
Surname=widgets.Text(description="Surname")
Age=widgets.IntSlider(min=0, max=100, value=50, description="Age")
display.display(Name, Surname, Age)

LabelDate = widgets.Text(value= "Today's date", disabled=True)
LabelDate.background_color = "#FFFFFF"
LabelDate.margin = "20px"
display.display(LabelDate)

Day=widgets.IntSlider(min=1, max=31, value=1, description="Day")
Month=widgets.IntSlider(min=1, max=12, value=1, description="Month")
Year=widgets.IntSlider(min=2016, max=2050, value=2016, description="Year")
display.display(Day, Month, Year)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from pynq.pmods import Grove_EarHR
import time
from IPython import display
import numpy as np

#sampling HR changes

hr=Grove_EarHR(1,4)
valueshr=[]
valuesBeats=[]
timeline=[]
startT=0
startBeat=0

while(True):
    try:
        beats, deltaT = hr.read_raw()
        print(deltaT)
        display.clear_output(wait=True)
        if deltaT!=0:
            while startT==0:
                startT=time.time()
                startBeat=beats
            timeline.append(time.time()-startT)
            valueshr.append(60000/deltaT)
            valuesBeats.append(beats-startBeat) 
            time.sleep(0.05)
    except KeyboardInterrupt:
        break
        
plt.plot(timeline, valueshr)   

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from IPython import display

beatTimes=[]
beatStart = -1
for i in range(len(timeline)):
    if(beatStart != valuesBeats[i]):
        beatTimes.append(timeline[i])
    beatStart = valuesBeats[i]    
    
medint=list()
medfreq=list()
beatTimesmed=list()

for i in range(len(beatTimes)-3):
    somma=(beatTimes[i+3]-beatTimes[i+2])+(beatTimes[i+2]-beatTimes[i+1])+(beatTimes[i+1]-beatTimes[i])
    new=somma/3
    medint.append(new)
    medfreq.append(60/new)
    beatTimesmed.append((beatTimes[i+3]+beatTimes[i+2]+beatTimes[i+1]+beatTimes[i])/4)

In [ ]:
#remotion of the outliers from the HR plot
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display
import numpy as np

medfreq2=list()

for i in medfreq:
    medfreq2.append(i)
    

def removeoutliers(dati, dati2):
    dati2.sort()
    posq1=(round(1+0.25*(len(dati2)-1)))-1
    posq3=(round(1+0.75*(len(dati2)-1)))-1
    q1=dati2[posq1]
    q3=dati2[posq3]
    IQR=q3-q1
    outliersq1=q1-1.5*IQR
    outliersq3=q3+1.5*IQR 
    datinp=np.array(dati)
    dati[1]=np.average(dati)
    for i in range(len(dati)):
        if dati[i]>=outliersq3 or dati[i]<=outliersq1:
            dati[i]=dati[i-1]

removeoutliers(medfreq, medfreq2)

In [ ]:
import os
import pickle

#creation of the user's folder


HRmax=max(medfreq)
HRmin=min(medfreq)
userFolder = "batticuore/" + Name.value + "_" + Surname.value + "/"
filename = userFolder + str(Day.value) + "_" + str(Month.value) + "_" + str(Year.value) + ".pkl"

os.makedirs(userFolder, exist_ok=True)

#user's information saved in dictionary

dizionario={}
dizionario["Name"]=Name.value
dizionario["Surname"]=Surname.value
dizionario["Age"]=Age.value
dizionario["Max HR"]=HRmax
dizionario["Min HR"]=HRmin

In [ ]:
medfreqnp=np.array(medfreq)
M=np.average(medfreq)

#comparison with standard values

def save_obj(filename, obj):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)

def load_obj(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

def checkfreq(mean, minim, mass):
    if mean>=minim and mean<=mass:
        return True
    else:
        return False

readage=Age.value

if readage<12:
    checkfreq(M, 80, 100)
    plt.plot([0, 1000], [80, 80], 'r')
    plt.plot([0, 1000], [100, 100], 'r')
elif readage>=12 and readage<18:
    checkfreq(M, 70, 120)
    plt.plot([0, 1000], [70, 70], 'r')
    plt.plot([0, 1000], [120, 120], 'r')
elif readage>=18 and readage<55:
    checkfreq(M, 60, 90)
    plt.plot([0, 1000], [60, 60], 'r')
    plt.plot([0, 1000], [90, 90], 'r')
elif readage>=55:
    checkfreq(M, 70, 90)
    plt.plot([0, 1000], [70, 70], 'r')
    plt.plot([0, 1000], [90, 90], 'r')
    
save_obj(filename, dizionario)  

plt.plot(beatTimesmed, medfreq, 'k')
plt.axis([0, max(beatTimesmed), 0, 130])
plt.xlabel("time in s")
plt.ylabel("HR in bpm")
plt.grid(True)
plt.title("grafico HR")
plt.show()

In [ ]:
#HR variability plot
deltachange=list()

for i in range(len(beatTimes)-1):
    deltachange.append(beatTimes[i+1]-beatTimes[i])
    
ascisse=np.arange(1, len(beatTimes), dtype=None)

deltachange2=list()

for i in deltachange:
    deltachange2.append(i)
    
removeoutliers(deltachange, deltachange2)

plt.plot(ascisse, deltachange)

In [ ]:
#suggestions in dictionary
if Age.value<=12:
    if checkfreq(M, 80, 100)==True:
        dizionario["Suggestions"]="The average of your HR fits the standard values. Feel free to do sport"
    else:    
        dizionario["Suggestions"]="The average of your HR doesn't fit the standard values. Be carefull!\
    You shouldn't train unless your doctor gives you permission."

elif Age.value>=12 and Age.value<=18:
    if checkfreq(M, 70, 120)==True:
        dizionario["Suggestions"]="The average of your HR fits the standard values. Feel free to do sport"
    else:    
        dizionario["Suggestions"]="The average of your HR doesn't fit the standard values. Be carefull!\
    You shouldn't train unless your doctor gives you permission."    
    
elif Age.value>=18 and Age.value<=55:
    if checkfreq(M, 60, 90)==True:
        dizionario["Suggestions"]="The average of your HR fits the standard values. Feel free to do sport"
    else:    
        dizionario["Suggestions"]="The average of your HR doesn't fit the standard values. Be carefull!\
    You shouldn't train unless your doctor gives you permission."  

    
elif Age.value>=55:
    if checkfreq(M, 70, 90)==True:
        dizionario["Suggestions"]="The average of your HR fits the standard values. Feel free to do sport"
    else:    
        dizionario["Suggestions"]="The average of your HR doesn't fit the standard values. Be carefull!\
    You shouldn't train unless your doctor gives you permission."      


In [ ]:
#pulse oximeter code

   

%matplotlib inline

   

from pynq.pmods import PMOD_ADC

   

import time

   

import matplotlib.pyplot as plt

   

import numpy as np

   

pmod_adc = PMOD_ADC(2)

   

value2= pmod_adc.read(1)

   

value = pmod_adc.read(0)

   

start = time.time()

   

end = start

   

samplingTime = 1 #setting this parameter (for ex samplingTime=10) you will obtain several repetitions of signal

   

#create two empty lists: one for redled, one for irled values

   

redled = []

   

irled = []

   

   

#acquire alternatively signals from red or ir led and append the values in correspondant list

   

while end - start < samplingTime:

   

   value = pmod_adc.read(0)

   

   redled.append(value)

   

   value = pmod_adc.read(1)

   

   irled.append(value)

   

   time.sleep(0.05)

   

   end = time.time()

   

   

#time from beginning to the end of sampling

   

deltaT = end - start

   

#samples number

   

samples = len(redled)

   

#frequency

   

freq = samples / deltaT

   

   

   

print("frequency: " + str(freq))

   

print("Voltage values ir led: " + str(irled))

   

print("Voltage vales red led: " + str(redled))

   

   

   

#create redled voltage plot   

   

plt.plot(range(len(redled)), redled, 'r')

   

plt.title('Red Led voltage')

plt.xlabel('Samples')

plt.ylabel('Amplitude')

plt.subplots_adjust(left=.05,right=.98)

plt.show()

   

   

#create irled voltage plot

   

plt.plot(range(len(irled)),irled,'r')

plt.title('Ir Led voltage')

plt.xlabel('Samples')

plt.ylabel('Amplitude')

plt.subplots_adjust(left=.05,right=.98)

plt.show()

   

   

   

   

#redled values average

   

avg = sum(redled)/len(redled) 

   

#create two empty list for min from red led list and max from redled list

   

minr= []

   

maxr= []

   

   

#find all min and max in redled list and save them in minr or maxr lists

   

maxL = avg 

   

minL = avg

   

   

if redled[0] > avg: 

   

    search = 1

   

else:

   

    search = -1

   

for val in redled: 

   

    if(val > avg): 

   

        if search != 1: 

   

            minr.append(minL)

   

            maxL = avg

   

            minL = avg

   

        search = 1

   

        maxL = max(maxL, val) 

   

    elif(val < avg):

   

        if search != -1:

   

            maxr.append(maxL)

   

            maxL = avg

   

            minL = avg

   

        search = -1

   

        minL = min(minL, val)

   

        

   

print("minr list is: " + str(minr))

   

print("maxr list is :" +s tr(maxr))

   

#irled values average 

   

avg = sum(irled)/len(irled) 

   

#create empty list for min from irled list and max from irled list

   

minir= []

   

maxir= []

   

#find all min and max in irled list and save them in minir or maxir list   

   

maxL = avg 

   

minL = avg

   

   

   

if irled[0] > avg: 

   

    search = 1

   

else:

   

    search = -1

   

for val in irled: 

   

    if(val > avg): 

   

        if search != 1: 

   

            minir.append(minL) 

   

            maxL = avg 

   

            minL = avg

   

        search = 1 

   

        maxL = max(maxL, val) 

   

    elif(val < avg):

   

        if search != -1:

   

            maxir.append(maxL)

   

            maxL = avg

   

            minL = avg

   

        search = -1

   

        minL = min(minL, val)

   

   

   

print(minir)

   

print(maxir)

   

   

#def R index function    

    

def R(a, c, ai, ci):

   

    h=(a/c)/(ai/ci)

   

    return h

   

   

   

g=R(max(redled), min(redled), max(irled), min(irled))

   

   

#R middle value

   

avgmaxr=sum(maxr)/len(maxr)

avgminr=sum(minr)/len(minr)

avgmaxir=sum(maxir)/len(maxir)

avgminir=sum(minir)/len(minir)

   

Rmed=R(avgmaxr, avgminr, avgmaxir, avgminir)

   

   

  

print('We measured your middle R index: ' +str(Rmed))

print("While your max R index is:" + str(g))

   

   

   

if(g >1.2) :

   

    print('Be careful! The oxigen level in your blood is not enough: according to you R index, your oxigen saturation is less than 85% !')

   

else:

   

    print('Ok! The oxigen level in your blood is optimal: according to your R index, it is over 90% ') 

    

   

   



   
 
   

   
